## Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.3 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow-text==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.1.21
    Uninstalling flatbuffers-23.1.21:
      Successfully uninstalled flatbuffers-23.1.21
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-esti

In [ ]:
import pandas as pd                                                        #data import
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer                  #character tokenization
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences          #padding
from tqdm import tqdm                                                      #track progress
import numpy as np                                                         #for numpy operations
from sklearn.model_selection import train_test_split                       #to split the data into train and test sets
from copy import deepcopy
from numpy import save, load
from transformers import BertTokenizer
import tensorflow_hub as hub
import tensorflow_text as text
import pickle
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# tokenizer.tokenize("I have a new GPU! semi-optimal")

In [ ]:
df = pd.read_csv("train_data.csv")  #upload train_data.csv provided in the runtime session or give path the file
#df2 = pd.read_csv('ModData.csv')

df = df.drop("Unnamed: 0",axis = 1)
Answer_end=[]
for i, row in df.iterrows():
    if df.at[i,'Answer_possible'] == False:
        df.at[i,'Answer_start'] = '[0]'
    df.at[i,'Answer_start'] = int(df.at[i,'Answer_start'][1:-1])
    Answer_end.append(df.at[i,'Answer_start']+len(df.at[i,'Answer_text'][2:-2]))
df["Answer_end"] = Answer_end
# df = df.head(3000)
# df.to_csv("data1.csv")

In [ ]:
# df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/data1.csv").drop('Unnamed: 0',axis=1)

In [ ]:
df.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,Answer_end
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],526,530
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],505,524
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],360,374
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],276,286
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],290,301


In [ ]:
#df2 = df2.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)

KeyError: ignored

In [ ]:
df = df.append(df2,ignore_index=True)
len(df)

78055

In [ ]:
df.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,Answer_end
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],526,530
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],505,524
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],360,374
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],276,286
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],290,301


In [ ]:
df.head()

In [ ]:
# df_t = df.head(200)
# df_t

# For Testing

In [ ]:
print('Possible ans- ' + str(len(df[df['Answer_possible']==True])))
print('Possible ans- ' + str(len(df[df['Answer_possible']==False])))

Possible ans- 50125
Possible ans- 27930


## Word Tokenization of Question and Context




In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [ ]:
# tokenizer.encode("[CLS] The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations. [SEP] What organization is the IPCC a part of? [SEP]")

In [ ]:
def find(context,ans):
  context = list(context)
  if len(ans) < 5:
    try:
      for idx in range(len(context)-len(ans)+1):
        if context[idx: idx + len(ans)] == ans:
          # print(idx)
          return idx
    except:
      print(tokenizer.decode(ans))
      print(tokenizer.decode(context))
      return -404
  else:
    try:
      for idx in range(len(context)-len(ans)+1):
        if context[idx+1: idx + len(ans)-1] == ans[1:-1]:
          # print(idx)
          return idx-1
    except:
      print(tokenizer.decode(ans))
      print(tokenizer.decode(context))
      return -404

In [ ]:
def tokenize_entries(row):
    """Function that tokenizes(both word and character level) the question and context of all records"""
    # row['word_tokens'] = tokenizer(row['Paragraph'],row['Question'],max_length=512,truncation='longest_first')
    ctok = tokenizer(row['Paragraph'])
    stop = [' ','.','-','!','?','\n']
    m = 1000000
    start = 0
    end = 0
    
    for x in stop:
      try:
        m = min(m,row['Paragraph'][row['Answer_end']:].index(x))
        # print(row['Paragraph'][row['Answer_start']:row['Answer_end']+m])
      except:
        pass
        
    ans = tokenizer(row['Paragraph'][row['Answer_start']:row['Answer_end']+m])
    ans=list(ans['input_ids'])[1:-1]
    # print(ans)
    try:
      start = 0 if not row["Answer_possible"] else find(ctok['input_ids'],ans)
      end = 0 if not row["Answer_possible"] else int(start)+len(list(ans))
    except:
      start = -404 if row["Answer_possible"] else 0
      end = -404 if row["Answer_possible"] else 0
    
    if end > 460:
        row['word_tokens'] = ctok
        row['Answer_start'] = -404
        return row
    qtok = tokenizer(row['Question'])

    ctok['input_ids'] = ctok['input_ids'] + [0]*461
    ctok['input_ids'] = ctok['input_ids'][0:460]+[120]
    ctok['attention_mask'] = ctok['attention_mask'] + [0]*461
    ctok['attention_mask'] = ctok['attention_mask'][0:460]+[1]
    ctok['token_type_ids'] = ctok['token_type_ids'] + [0]*461
    ctok['token_type_ids'] = ctok['token_type_ids'][0:461]

    # print(len(ctok['token_type_ids']))
    # print(len(ctok['input_ids']))
    # print(len(ctok['attention_mask']))

    # print(qtok['input_ids'][0])
    qtok['input_ids'] = qtok['input_ids'][1:] + [0]*51
    qtok['input_ids'] = qtok['input_ids'][0:51]
    # print(qtok['input_ids'][0])
    qtok['attention_mask'] = qtok['attention_mask'][1:] + [0]*51
    qtok['attention_mask'] = qtok['attention_mask'][0:51]
    qtok['token_type_ids'] = [1]*51

    # print(len(qtok['token_type_ids']))
    # print(len(qtok['input_ids']))
    # print(len(qtok['attention_mask']))

    ctok['input_ids'] = ctok['input_ids'] +qtok['input_ids']
    ctok['attention_mask'] = ctok['attention_mask'] + qtok['attention_mask'] 
    ctok['token_type_ids'] = ctok['token_type_ids'] + qtok['token_type_ids'] 

    # print(len(ctok['token_type_ids']))
    # print(len(ctok['input_ids']))
    # print(len(ctok['attention_mask']))
    row['word_tokens'] = ctok
    
    # row['paragraph_id'] = 

    row['Answer_start'] = start  if row["Answer_possible"] else 0 
    row['Answer_end'] = end  if row["Answer_possible"] else 0
    return row

In [ ]:
tqdm.pandas()

In [ ]:
df_final = df.progress_apply(tokenize_entries, axis=1,)

100%|██████████| 78055/78055 [10:06<00:00, 128.71it/s]


In [ ]:
len(df_final[df_final['Answer_end'] > 512])

6

In [ ]:
len(df_final[df_final['Answer_start'] == -404]) #Drop These

35

In [ ]:
df_final = df_final[df_final['Answer_start'] != -404]
df_final = df_final[df_final['Answer_start'] < 512]

In [ ]:
print('Possible ans- ' + str(len(df[df['Answer_possible']==True])))
print('Possible ans- ' + str(len(df[df['Answer_possible']==False])))

Possible ans- 50125
Possible ans- 27930


In [ ]:
print('Possible ans- ' + str(len(df_final[df_final['Answer_possible']==True])))
print('Possible ans- ' + str(len(df_final[df_final['Answer_possible']==False])))

Possible ans- 50090
Possible ans- 27930


## Train Test Split

In [ ]:
df_train, df_test = train_test_split(df_final, test_size = 0.2,random_state=100)

In [ ]:
print(df_train.shape, df_test.shape)

(62416, 8) (15604, 8)


In [ ]:
#dataset after tokenization
df_train.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,Answer_end,word_tokens
55602,Madrasa,"People of all ages attend, and many often move...",What age groups are never enrolled in madaris?,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
33077,Florida,"From 1952 to 1964, most voters were registered...",What act made it illegal for minorities to vote?,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
57817,Canadian_football,The Grey Cup was established in 1909 after bei...,When did amateur teams begin to dominate the C...,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
65826,Windows_8,"However, according to research firm NPD, sales...","While Windows sales went down, which company e...",True,['Apple'],50,51,"[input_ids, token_type_ids, attention_mask]"
25627,Nintendo_Entertainment_System,A thriving market of unlicensed NES hardware c...,What was the name of the SNES clone produced ...,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"


In [ ]:
tokenizer.decode(df_train.iloc[0]['word_tokens']['input_ids'][df_train.iloc[0]['Answer_start']:df_train.iloc[0]['Answer_end']])

''

In [ ]:
#dataset after padding
df_train.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,Answer_end,word_tokens
55602,Madrasa,"People of all ages attend, and many often move...",What age groups are never enrolled in madaris?,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
33077,Florida,"From 1952 to 1964, most voters were registered...",What act made it illegal for minorities to vote?,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
57817,Canadian_football,The Grey Cup was established in 1909 after bei...,When did amateur teams begin to dominate the C...,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"
65826,Windows_8,"However, according to research firm NPD, sales...","While Windows sales went down, which company e...",True,['Apple'],50,51,"[input_ids, token_type_ids, attention_mask]"
25627,Nintendo_Entertainment_System,A thriving market of unlicensed NES hardware c...,What was the name of the SNES clone produced ...,False,[],0,0,"[input_ids, token_type_ids, attention_mask]"


In [ ]:
test_input_mask,test_input_ids,test_input_type_ids = [],[],[]
train_input_mask,train_input_ids,train_input_type_ids = [],[],[]
for row in df_train['word_tokens'].values:
  train_input_mask.append(row['attention_mask'])
  train_input_ids.append(row['input_ids'])
  train_input_type_ids.append(row['token_type_ids'])
for row in df_test['word_tokens'].values:
  test_input_mask.append(row['attention_mask'])
  test_input_ids.append(row['input_ids'])
  test_input_type_ids.append(row['token_type_ids'])

In [ ]:
train_input_mask = np.array(train_input_mask,dtype = np.int32)
train_input_ids = np.array(train_input_ids,dtype = np.int32)
train_input_type_ids = np.array(train_input_type_ids,dtype = np.int32)
test_input_mask = np.array(test_input_mask,dtype = np.int32)
test_input_ids = np.array(test_input_ids,dtype = np.int32)
test_input_type_ids = np.array(test_input_type_ids)

In [ ]:
train_input_type_ids [0]

## Creating Output Arrays

In [ ]:
#creating output arrays
num_classes = 461 #PreCalculated for Bert large
y_start_train = tf.keras.utils.to_categorical(df_train['Answer_start'].values, num_classes)
y_end_train = tf.keras.utils.to_categorical(df_train['Answer_end'].values, num_classes)

y_start_test = tf.keras.utils.to_categorical(df_test['Answer_start'].values, num_classes)
y_end_test = tf.keras.utils.to_categorical(df_test['Answer_end'].values, num_classes)

## Saving all the required variables to disk

In [ ]:
# Saving Train Data
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/mask_trainna.npy", train_input_mask)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/ids_trainna.npy", train_input_ids)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/type_trainna.npy", train_input_type_ids)

In [ ]:
#saving test input data
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/mask_testna.npy", test_input_mask)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/ids_testna.npy", test_input_ids)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/type_testna.npy", test_input_type_ids)

In [ ]:
#saving output data
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_start_trainna.npy", y_start_train)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_end_trainna.npy", y_end_train)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_start_testna.npy", y_start_test)
save("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_end_testna.npy", y_end_test)